DP算法通常步骤：
1. 刻画一个最优解的结构特征
2. 递归定义最优解的值
3. 计算最优解的值，通常采用自底向上的方法
4. 利用计算出的信息构造一个最优解

## 15.1 钢条切割
**问题描述：**
给定一段长度为 n 英寸的钢条和一个价格表 p<sub>i(i = 1,2,...,n),求切割钢条方案，使得销售收益 r<sub>n 最大。
    
钢条切割问题满足最优子结构性质：问题的最优解又相关子问题的最优解组合而成，而这些子问题可以独立求解。



*自顶向下递归求解（非动态规划）：*

In [7]:
import math
def cut(p,n):
    if n == 0:
        return 0
    q = -math.inf
    for i in range(1,n+1):
        q = max(q,p[i]+cut(p,n-i))
    return q   

以上方法的时间复杂度为 2<sup>n  



动态规划有两种等价的实现方法  
第一种方法称为**带备忘的自顶向下法**。此方法仍按自然的递归形式编写过程，但过程会保存每个子问题的解（数组/散列表）。当需要一个子问题的解时，过程首先检查是否已经保存过此解。如果是，则直接返回保存的值，从而节省了计算时间；否则，按通常方式计算这个子问题。  
第二种方法称为**自底向上法**。这种方法一把需要恰当定义子问题“规模”的概念，使得任何子问题的求解都只依赖于“更小的”子问题的求解。因而我们可以将子问题按规模排序，按由小至大的顺序进行求解。当求解某个子问题时，它所依赖的那些更小的子问题都已求解完毕，结果已经保存。每个子问题只需求解一次，当我们求解它时，它的所有前提子问题都已求解完成。  


带备忘的自顶向下解法：

In [1]:
import math
def memocut(p,n):
    r = [-math.inf]*(n+1)
    return memocutaux(p,n,r)
    

def memocutaux(p,n,r):
    if r[n]>=0:
        return r[n]
    if n == 0:
        q = 0
    else:
        q = -math.inf
        for i in range(1,n+1):
            q = max(q,p[i]+memocutaux(p,n-i,r))
    r[n] = q
    return q


自底向上解法：

In [ ]:
import math
def bottomToUp(p,n):
    r = [-math.inf]*(n+1)
    r[0] = 0
    for j in range(1,n+1):
        q = -math.inf
        for i in range(1,j+1):
            q = max(q,p[i]+r[j-i])
        r[j] = q
    return r[n]



计算最大收益值同时保存最优解对应的第一段钢条切割长度：

In [1]:
import math
def eXBottomUp(p,n):
    r = [-math.inf]*(n+1)
    s = [0]*(n+1)
    for j in range(1,n+1):
        q = -math.inf
        for i in range(1,j+1):
            if q < p[i]+r[j-i]:
                q = p[i]+r[j-i]
                s[j] = i
        r[j] = q
    return [r[n],s[n]]

  
***练习***   
  
15.1-3 对钢条切割问题进行修改，除了切割下的钢条段具有不同价格p<sub>i外，每次切割还要付出固定成本c  
    

In [4]:
import math
def modiCut(p,n,c):
    r = [-math.inf]*(n+1)
    r[0] = 0
    for j in range(1,n+1):
        q = -math.inf
        for i in range(1,j+1):
            if i == j:
                q = max(q,p[i])
            else:
                q = max(q,p[i]+r[j-i]-c)
        r[j] = q
    return r[n]



## 15.4 最长公共子序列  

**最长公共子序列问题**：  
给定两个序列 X=(x<sub>1</sub> ,x<sub>2</sub> ,...,x<sub>m</sub> )和 Y=(y<sub>1</sub> ,y<sub>2</sub> ,...,y<sub>n</sub> )，求 X 和 Y 长度最长的公共子序列  
    
令 X=(x<sub>1</sub> ,x<sub>2</sub> ,...,x<sub>m</sub> )和 Y=(y<sub>1</sub> ,y<sub>2</sub> ,...,y<sub>n</sub> )为两个序列，Z=(z<sub>1 </sub>,z<sub>2</sub> ,...z<sub>k</sub> )为 X 和 Y 的任意 LCS。  
    
1. 如果 x<sub>m</sub> = y<sub>n</sub>,则z<sub>k</sub>=x<sub>m</sub>=y<sub>n</sub> 且 Z<sub>k-1</sub> 是 X<sub>m-1</sub> 和 Y<sub>n-1</sub> 的一个 LCS
2. 如果 x<sub>m</sub>$\neq$y<sub>n</sub>, 那么z<sub>k</sub>$\neq$x<sub>m</sub> 意味着 Z 是 X<sub>m-1</sub> 和 Y 的一个 LCS
3. 如果 x<sub>m</sub>$\neq$y<sub>n</sub>, 那么z<sub>k</sub>$\neq$y<sub>n</sub> 意味着 Z 是 X 和 Y<sub>n-1</sub> 的一个 LCS

$$ c[i,j]=
\begin{cases}
0& \ i = 0 \ or \ j = 0\\
c[i-1,j-1] + 1& \ i,j>0 \ and \ x_i = y_j\\
max(c[i,j-1],c[i-1,j])& \ i,j>0 \ and \ x_i \ne x_j
\end{cases}$$

In [14]:
def lcsLength(X:str,Y:str):
    m = len(X)
    n = len(Y)
    X=" "+X
    Y=" "+Y
    b = [[0] * (n+1) for num in range(m+1)]
    c = [[0] * (n+1) for num in range(m+1)]
    for i in range(1,m+1):
        c[i][0] = 0
    for j in range(0,n+1):
        c[0][j] = 0
    for i in range(1,m+1):
        for j in range(1,n+1):
            if X[i] == Y[j]:
                c[i][j] = c[i-1][j-1]+1
                b[i][j] = "left-up"
            elif c[i-1][j] >= c[i][j-1]:
                c[i][j] = c[i-1][j]
                b[i][j]="up"
            else:
                c[i][j] = c[i][j-1]
                b[i][j]="left"
    return (b,c)

def printLCS(result:tuple,X,i,j):
    b = result[0]
    if i == 0 or j == 0:
        return 
    if b[i][j] == "left-up":
        printLCS(result,X,i-1,j-1)
        print(X[i-1])
    elif b[i][j] == "up":
        printLCS(result,X,i-1,j)
    else:
        printLCS(result,X,i,j-1)

test1 = "AABBDC"
test2 = "DDBDCA"
result = lcsLength(test1,test2)
printLCS(result,test1,6,6)

B
D
C


***练习***  

15.4-5 设计一个O(n<sup>2</sup>)时间的算法，求一个 n 个数的序列的最长单调递增子序列

${思路： \\ 假设 \ M_i \ 是 \ N_j \ 的最长单调递增子序列长度， \ m_k \ 是序列 \ M_i \ 一个元素}$ 
$$
M_i= 
\begin{cases}
0& \ j = 0\\
max(M_k)+1 & \ \exists k, \ N_j>m_k\\
1 & \ \forall k, \ N_j \le m_k
\end{cases}
$$

In [9]:
def monotonic(s):
    length = len(s)
    alpha = [0 for col in range(length)]
    ms = 0
    mp = 0
    seq = []
    for i in range(0,length):
        if i == 0:
            alpha[i] = [-1,1]
        else:
            maxseq = 0
            temppos = 0
            for k in range(0,i):
                if s[i] > s[k]:
                    if alpha[k][1] > maxseq:
                        maxseq = alpha[k][1]
                        temppos = k
                    maxseq = max(maxseq,alpha[k][1])
            if maxseq != 0:
                alpha[i] = [temppos,maxseq+1]
                if ms < maxseq+1:
                    ms = maxseq+1
                    mp = i
            else:
                alpha[i] = [0,1]
    point = mp
    while point!= -1:
        seq = [s[point]] + seq
        point = alpha[point][0]
    return seq


s = [1,3,2,3]
monotonic(s)

[1, 2, 3]

## 思考题

15-2. 求给定字符串的最长回文子序列

${思路：\\
 p[i][j] \ 为字符串 \ s[i:j+1] \ 中回文子序列长度，则有}$
$$
p[i][j]= 
\begin{cases}
0 & \ i > j\\
1 & \ i = j\\
p[i+1][j-1]+2 & \ s[i] = s[j] \ \& \ i<j\\
max(p[i+1][j],p[i][j-1]) & \ s[i] \ne s[j]
\end{cases}
$$
 

In [47]:
def LPS(s):
    if not s:
        return 0
    length = len(s)
    p = [[0 for col in range(length)] for row in range(length)]
    d = 0
    answer = 0
    while d < length:
        p[d][d] = 1
        d += 1
    for j in range(1,length):
        for i in range(j-1,-1,-1):
            if s[i] == s[j]:
                p[i][j] = p[i+1][j-1]+2
            else:
                p[i][j] = max(p[i+1][j],p[i][j-1])
            answer = max(answer,p[i][j])
    return answer if answer > 1 else 1

s = "character"
LPS(s)

5

变种：求给定字符串的最长回文子串

In [48]:
def LPCS(s):
    if not s:
        return 0
    length = len(s)
    p = [[0 for col in range(length)] for row in range(length)]
    d = 0
    answer = 1
    row = 0
    col = 0
    while d < length:
        p[d][d] = 1
        d += 1
    for j in range(1,length):
        for i in range(j-1,-1,-1):
            if s[i] == s[j]:
                p[i][j] = p[i+1][j-1]+2 if p[i+1][j-1] > 0 else 0
            else:
                p[i][j] = 0
            if p[i][j] > answer:
                answer = p[i][j]
                row = i
                col = j
    return s[row:col+1]

s = "character"
LPCS(s)

'ara'

15-4. 用打印机打印一段文本。输入文本为 $ \ n \ $个单词的序列，单词长度分别为 $ \ l_1,l_2,l_3,...,l_n \ $ 个字符。每行最多$ \ M \ $ 个字符。如果某行包含第 $ \ i \  $ 到第 $ \ j \ (i \le j) $ 个单词，且单词间隔为一个空格符，则行尾的额外空格符数量为 $ M-j+i-\sum_{k=j}^il_k $ ，此值必须为非负，否则一行无法容纳这些单词。设计算法使得所有行的（除最后一行外）额外空格数的立方之和最小。  
  

In [42]:
import math
def neat(a,M):
    length = len(a)
    W = [[-1 for col in range(length)] for row in range(length)]
    for i in range(length):
        for j in range(i,length):
            space = M - j + i-sum(a[i:j+1])
            W[i][j] = space**3 if space >= 0 else -1
    B = [math.inf]*length
    C = [-1]*length
    B[0] = W[0][0]
    t = 1
    while t<length:
        for k in range(0,t+1):
            if k == 0 and W[k][t] != -1:
                B[t] = min(W[k][t],B[t])
                if B[t] == W[k][t]:
                    C[t] = -1
            if W[k][t] != -1:
                B[t] = min(B[k-1]+W[k][t],B[t])
                if B[t] == B[k-1]+W[k][t]:
                    C[t] = k-1
        t += 1
    print(B)
    print(C)
    return W

a = [3,1,2,6,2,4,3,6,5,2,4]
M = 8
neat(a,M)
        

[125, 27, 0, 8, 224, 9, 134, 142, 169, 142, 170]
[-1, -1, -1, 2, 3, 3, 5, 6, 7, 7, 8]


[[125, 27, 0, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, 343, 64, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, 216, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, 8, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, 216, 1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, 64, 0, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, 125, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, 8, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, 27, 0, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, 216, 1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 64]]

15-5. 编辑距离  
[力扣编辑距离](https://leetcode-cn.com/problems/edit-distance/)的复杂化,在此只贴上力扣题目的代码

In [3]:
import math
def minDistance(word1: str, word2: str):
    word1 = " " + word1
    word2 = " " + word2
    m = len(word1)
    n = len(word2)
    dp = [[0 for col in range(m)]for row in range(n)]
    for i in range(n):
        dp[i][0] = i
    for j in range(m):
        dp[0][j] = j
    dp[0][0] = 0
    for i in range(1,n):
        for j in range(1,m):
            mini = 0
            if word1[j] == word2[i]:
                mini = min(dp[i-1][j],dp[i][j-1],dp[i-1][j-1]-1)
            else:
                mini = min(dp[i-1][j],dp[i][j-1],dp[i-1][j-1])
            dp[i][j] = mini+1
    return dp[-1][-1]

a = "zoologicoarchaeologist"
b = "zoogeologist"
minDistance(a,b)

10

15-6.根据题意归纳，即通过给定二叉树，选择节点，使得选取节点之和最大，且各节点之间不存在父子关系

In [ ]:
class lcrs:
    def __init__(self,x=None):
        self.value = x
        self.leftchild = None
        self.rightsibling = None